In [1]:
from csoundengine import *

In [2]:
s = Session()

using libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [3]:
s.defInstr('busoscil', r'''
|ibus|
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(busoscil, ibus=0)

In [4]:
bus = s.assignBus(value=1000)
synth = s.sched('busoscil', ibus=bus.token)
automation = bus.automate((0, 800, 3, 500), overtake=False)
synth

clearbuses -------------------


Button(description='Stop', style=ButtonStyle())

Output()

Synth(‖ busoscil=401.0002 start=11.011 dur=-1 ibus=0)

In [5]:
s.engine.unsched(automation)

In [6]:
synth.stop()

In [7]:
import sndfileio
import time
t0 = time.time()
info = sndfileio.sndinfo('../snd/piano-pitchtest-2.flac')
t1 = time.time()
print(info, t1 - t0)

samplerate : 44100
nframes    : 459938
channels   : 1
encoding   : pcm16
fileformat : flac
duration   : 10.429 0.0011982917785644531


In [8]:
tab = s.readSoundfile('../snd/piano-pitchtest-2.flac')
print(t1 - t0)
s.defInstr('test0', r'''
kfreq = p5
outch 1, oscili:a(0.1, lag(kfreq, 0.01))
''')
s.sched('test0', 0, 2, kfreq=440)
ev = s.playSample(tab)
ev

0.0011982917785644531


Button(description='Stop', style=ButtonStyle())

Output()

Synth(‖ .playSample=403.0002 start=21.82 dur=10.449 isndtab=301 istart=0 ifadein=0.02 ifadeout=0.02 kchan=1 kspeed=1 kgain=1 kpan=0.5 ixfade=-1)

In [9]:
s.defInstr('test1', r'''
|kfreq|
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test1, kfreq=0)

In [10]:
name = 'test5'
s.defInstr(name, r'''
ibus = p5
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test5, ibus:5)

In [11]:
bus = s.assignBus(value=1000)
print(bus)
synth = s.sched(name, ibus=bus.token)
bus.automate((0, 800, 3, 500), overtake=False)
synth

Bus('control', token=1, bound=True)


Button(description='Stop', style=ButtonStyle())

Output()

Synth(‖ test5=404.0002 start=27.977 dur=-1 ibus=1)

In [13]:
synth.stop()
bus.release()

In [14]:
s.engine.busSystemStatus()

{'audioBusesFree': 512.0,
 'controlBusesFree': 512.0,
 'numAudioBuses': 64,
 'numControlBuses': 512}

In [16]:
e = s.engine

buses = [e.assignBus(value=600) for _ in range(200)]
e.sync()
for bus in buses:
    synth = s.sched('test5', dur=1, ibus=bus)
    e.writeBus(bus, 700, 0.001)
    

In [18]:
s.defInstr('test', r'''
  ibus = p5
  kfreq = busin:k(ibus)
  outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')

buses = [s.assignBus(value=2000) for _ in range(200)]
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus.token)
    bus.set(1000, 0)
    

In [24]:
e.writeBus(bus.token, 1500)

In [28]:
bus.set(1600)

In [18]:
e.unschedAll()

end of section 1	 sect peak amps:  0.87595  0.00000
SECTION 1:


In [4]:
s.defInstr('sender', r'''
ibus = p5
ifreqbus = p6
kfreq = busin:k(ifreqbus)
asig oscili 0.1, lag:k(kfreq, 0.01)
busout(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus  = p5
kgain = p6
asig = busin:a(ibus)
asig *= a(kgain)
outch 1, asig
''')


Instr(receiver, ibus:5, kgain:6)

In [5]:
bus = s.assignBus()
freqbus = s.assignBus(value=600)

chain = SynthGroup([s.sched('sender', ibus=bus, ifreqbus=freqbus),
                    s.sched('receiver', ibus=bus, kgain=0.5)])
# automate wins over set
freqbus.set(1500, delay=1.5)
freqbus.automate((0, 0, 1, 400), delay=1, overtake=True)

chain

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,p4,ibus,ifreqbus
401.0002 ‖,0.023,-1.000,0,2,3
p1,start,dur,p4,ibus,kgain
402.0002 ‖,0.013,-1.000,0,2,0.5


In [7]:
s.unschedAll()
s

Session(engine0, backend=jack, outdev=Built-in Audio Analog Stereo, synths=0)

In [1]:
from csoundengine import *
s = Session()

using libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [2]:
s.engine.hasBusSupport()

False

In [3]:
s.defInstr('sender', r'''
ibus = p5
asig oscili 0.1, 1000
busmix(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus = p5
asig = busin:a(ibus)
outch 1, asig
''')

Instr(receiver, ibus:5)

In [4]:
bus = s.assignBus('audio', persist=True)


In [5]:
s.engine.busSystemStatus()

{'audioBusesFree': 63,
 'controlBusesFree': 512,
 'numAudioBuses': 64,
 'numControlBuses': 512,
 'busIndexes': {}}

In [11]:
s0 = s.sched('sender', ibus=bus)
s1 = s.sched('receiver', ibus=bus, priority=2)
g = SynthGroup((s0, s1))
g

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,p4,ibus
401.0004 ‖,0.041,-1.000,0,-1
p1,start,dur,p4,ibus
901.0004 ‖,0.041,-1.000,0,-1



INIT ERROR in instr 401 (opcode _bususe) line 51: 
   Bus not found (token -1)


initerror #S7:S
		   T 27.299 - note deleted.  i401 had 1 init errors

INIT ERROR in instr 28 (opcode initerror.s) line 259: 
   Bus not found, token -1

initerror #S1:S
		   T 27.299 - note deleted.  i28 had 1 init errors

INIT ERROR in instr 901 (opcode _bususe) line 51: 
   Bus not found (token -1)


initerror #S7:S
		   T 27.300 - note deleted.  i901 had 1 init errors

INIT ERROR in instr 28 (opcode initerror.s) line 259: 
   Bus not found, token -1

initerror #S1:S
		   T 27.300 - note deleted.  i28 had 1 init errors


In [12]:
bus

Bus('audio', token=-1, bound=True)

In [13]:
bus.release()